# Практика по оптимизации гиперпораметров модели

Данные представлены в формате CSV.  Каждая строка представляет молекулу. 
- Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
- Остальные столбцы D1-D1776 представляют собой молекулярные **дескрипторы** — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

В качестве метрики будем использовать F1-score.

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

In [1]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import optuna

import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
%matplotlib inline
plt.style.use('seaborn')

ModuleNotFoundError: ignored

In [2]:
data = pd.read_csv('https://lms.skillfactory.ru/assets/courseware/v1/9f2add5bca59f8c4df927432d605fff3/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/_train_sem09__1_.zip')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data.tail()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
3746,1,0.033300,0.506409,0.10,0.0,0.209887,0.633426,0.297659,0.376124,0.727093,...,0,0,0,0,0,0,0,0,0,0
3747,1,0.133333,0.651023,0.15,0.0,0.151154,0.766505,0.170876,0.404546,0.787935,...,0,0,1,0,1,0,1,0,0,0
3748,0,0.200000,0.520564,0.00,0.0,0.179949,0.768785,0.177341,0.471179,0.872241,...,0,0,0,0,0,0,0,0,0,0
3749,1,0.100000,0.765646,0.00,0.0,0.536954,0.634936,0.342713,0.447162,0.672689,...,0,0,0,0,0,0,0,0,0,0
3750,0,0.133333,0.533952,0.00,0.0,0.347966,0.757971,0.230667,0.272652,0.854116,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data['Activity'].value_counts()

1    2034
0    1717
Name: Activity, dtype: int64

Условно можно сказать равномерно распределен целевой признак

In [5]:
X = data.drop('Activity', axis=1)
y = data['Activity']

In [6]:
# разделим выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3000, 1776) (751, 1776) (3000,) (751,)


# Logistic Regression without optimization

In [7]:
logis_reg = linear_model.LogisticRegression(
    max_iter=500,
    random_state=42
)
logis_reg.fit(X_train, y_train)
y_train_pred = logis_reg.predict(X_train)
print('f1 на обущающей выборке: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = logis_reg.predict(X_test)
print('f1 на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1 на обущающей выборке: 0.89
f1 на тестовой выборке: 0.78


# Random Forest without optimization

In [8]:
rf = ensemble.RandomForestClassifier(
    n_estimators=100,
    max_depth = 10,
    random_state=42
)
rf.fit(X_train, y_train)
y_train_pred = rf.predict(X_train)
print('f1 на обущающей выборке: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = rf.predict(X_test)
print('f1 на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1 на обущающей выборке: 0.94
f1 на тестовой выборке: 0.79


# GridSearchCV

## Logistic Regression

In [ ]:
param_grid = [
    {
        'penalty': ['l2', 'none'],
        'solver': ['lbfgs', 'sag'],
        'C': list(np.linspace(0.01, 1, 10, dtype=float))
    },
    {
        'penalty': ['l1', 'l2'] ,
        'solver': ['liblinear', 'saga'],
        'C': list(np.linspace(0.01, 1, 10, dtype=float))
    }
]

grid_searh = GridSearchCV(
    estimator=logis_reg,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1
)

%time grid_searh.fit(X_train, y_train)
y_train_pred = grid_searh.predict(X_train)
print('f1 на обущающей выборке c GridSearchCV: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = grid_searh.predict(X_test)
print('f1 на тестовой выборке c GridSearchCV: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

In [ ]:
print("Наилучшая модель:\n{}".format(grid_searh.best_estimator_))
print("Наилучшее значение точности при кросс-валидации: {:.2f}".format(grid_searh.best_score_))

In [ ]:
visual = pd.pivot_table(pd.DataFrame(grid_searh.cv_results_),
               values='mean_test_score', index='param_C',
               columns='param_solver')
sns.heatmap(visual)
plt.title('Тепловая карта зависимости метрики accuracy от solver и С') # подпись графика
sns.set(rc={'figure.figsize':(12, 8)}) #задаем размер графика

## Random Forest

In [ ]:
param_grid_1 = {'max_depth': [3, 5, 7, 9],
               'min_samples_leaf':[3, 5, 7, 9],
               'n_estimators': [100, 300, 500], 
               'criterion': ['gini', 'entropy']}

grid_search_1 = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_1,
    cv=5,
    n_jobs=-1
)

%time grid_search_1.fit(X_train, y_train)
y_train_pred = grid_search_1.predict(X_train)
print('f1 на тестовой выборке c GridSearchCV: {:.2f}'.format(metrics.f1_score(y_train_pred, y_train)))
y_test_pred = grid_search_1.predict(X_test)
print('f1 на тестовой выборке c GridSearchCV: {:.2f}'.format(metrics.f1_score(y_test_pred, y_test)))

In [ ]:
print("Наилучшая модель:\n{}".format(grid_search_1.best_estimator_))
print("Наилучшее значение точности при кросс-валидации: {:.2f}".format(grid_search_1.best_score_))

In [ ]:
visual_1 = pd.pivot_table(pd.DataFrame(grid_search_1.cv_results_),
               values='mean_test_score', index='param_C',
               columns='param_solver')
sns.heatmap(visual)
plt.title('Тепловая карта зависимости метрики accuracy от solver и С') # подпись графика
sns.set(rc={'figure.figsize':(12, 8)}) #задаем размер графика

# RandomizedSearchCV

## Logistic Regression

In [ ]:
param_grid_2 = [{'penalty': ['l2', 'none'],
              'solver': ['lbfgs', 'sag'],
              'C': list(np.linspace(0.01, 1, 10, dtype=float))},
              
              {'penalty': ['l1', 'none'],
              'solver': ['liblinear', 'saga'],
              'C': list(np.linspace(0.01, 1, 10, dtype=float))}
]

random_search = RandomizedSearchCV(
    estimator=logis_reg,
    param_distributions=param_grid_2,
    cv=5,
    n_iter=50,
    n_jobs=-1
)

random_search.fit(X_train, y_train)
%time random_search.fit(X_train, y_train)
y_train_pred = random_search.predict(X_train)
print('f1 на тестовой выборке c RandomizedSearchCV: {:.2f}'.format(metrics.f1_score(y_train_pred, y_train)))
y_test_pred = random_search.predict(X_test)
print('f1 на тестовой выборке c RandomizedSearchCV: {:.2f}'.format(metrics.f1_score(y_test_pred, y_test)))

## Random Forest

In [ ]:
param_grid_3 = {'max_depth': [3, 5, 7, 9],
               'min_samples_leaf': [3, 5, 7, 9],  #первый словарь 
               'n_estimators': [100, 300, 500], 
               'criterion': ['gini', 'entropy']}

random_search_1 = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid_3,
    cv=5,
    n_iter=50,
    n_jobs=-1
)
random_search_1.fit(X_train, y_train)
%time random_search_1.fit(X_train, y_train)
y_train_pred = random_search_1.predict(X_train)
print('f1 на тестовой выборке c RandomizedSearchCV: {:.2f}'.format(metrics.f1_score(y_train_pred, y_train)))
y_test_pred = random_search_1.predict(X_test)
print('f1 на тестовой выборке c RandomizedSearchCV: {:.2f}'.format(metrics.f1_score(y_test_pred, y_test)))

# Hyperopt

## Logistic Regression


In [ ]:
space = [{'penalty': hp.choice('penalty', {'l2', 'none'}),
     'solver': hp.choice('solver', {'lbfgs', 'sag'}),
     'C': hp.uniform('C', 0.01, 1, 10)}]

random_state = 42

def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': int(params['penalty']), 
              'solver': int(params['solver']), 
              'C': int(params['C'])
              }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score


trials = Trials() # используется для логирования результатов

best = fmin(hyperopt_rf, # наша функция 
            space=space, # пространство гиперпараметров
            algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
            max_evals=50, # максимальное количество итераций
            trials=trials, # логирование результатов
            rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
            )

print("Наилучшие значения гиперпараметров {}".format(best))

# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(
    random_state=random_state, 
    penalty=int(best['penalty']),
    solver=int(best['solver']),
    C=int(best['C'])
)

model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1 на обучающей выборке для  logistic regression с Hyperopt: {:.2f}'.format(metrics.f1_score(y_train_pred, y_train)))
y_test_pred = model.predict(X_test)
print('f1 на тестовой выборке для logistic regression с Hyperopt: {:.2f}'.format(metrics.f1_score(y_test_pred, y_test)))

## Random Forest

In [ ]:
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    # score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score


In [ ]:
%%time

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

In [ ]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

# Optuna

## Logistic Regression


In [ ]:
def optuna_rf(trial):
    # задаем пространства поиска гиперпараметров
    penalty = trial.suggest_categorical('penalty', {'l2', 'none'})
    solver = trial.suggest_categorical('solver', {'lbfgs', 'sag'})
    C = trial.suggest_float('C', 0.01, 1, step=10)
    
    # создаем модель
    model = linear_model.LogisticRegression(penalty=penalty,
                                           solver=solver,
                                           C=C,
                                           random_state=random_state)
    # обучаем модель
    model.fit(X_train, y_train)
    score = metrics.f1_score(y_train, model.predict(X_train))
    
    return score

# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз

study.optimize(optuna_rf, n_trials=50)

# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study.best_params,random_state=random_state)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
print('F1 на обучающей выборке для случайного леса с Optuna: {:.2f}'.format(metrics.f1_score(y_train_pred, y_train)))
y_test_pred = model.predict(X_test)
print('F1 на тестовой выборке для случайного леса с Optuna: {:.2f}'.format(metrics.f1_score(y_test_pred, y_test)))

## Random Forest

In [ ]:
def optuna_rf(trial):
    # задаем пространства поиска гиперпараметров
    n_estimators = trial.suggest_int('n_estimators', 100, 600, 200)
    max_depth = trial.suggest_int('max_depth', 3, 10, 2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 3, 7, 1)
    
    # создаем модель
    model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                            max_depth=max_depth,
                                            min_samples_leaf=min_samples_leaf,
                                            random_state=random_state)
    
    # обучаем модель
    model.fit(X_train, y_train)
    score = metrics.f1_score(y_train, model.predict(X_train))
    
    return score

# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=50)

# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
print('F1 с обучающей выборке для случайного леса с Optuna: {:.2f}'.format(metrics.f1_score(y_train_pred, y_train)))
y_test_pred = model.predict(X_test)
print('F1 тестовой выборке для случайного леса с Optuna: {:.2f}'.format(metrics.f1_score(y_test_pred, y_test)))